## Input: Kaggle Sqlite European Soccer Database 

## Output: R data frames persisted in csv files to facilitate easy Q&A

## Process: sqllite to R data frames to csv.  

1 : Load the libraries

In [1]:
library(dplyr)
library(purrr)
library(tidyr)
library(ggplot2)
library(broom)
library(magrittr)
library(RSQLite)
library(reshape2)
library(jsonlite)
library(knitr)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Attaching package: 'purrr'

The following objects are masked from 'package:dplyr':

    contains, order_by


Attaching package: 'magrittr'

The following object is masked from 'package:tidyr':

    extract

The following object is masked from 'package:purrr':

    set_names


Attaching package: 'reshape2'

The following object is masked from 'package:tidyr':

    smiths


Attaching package: 'jsonlite'

The following object is masked from 'package:purrr':

    flatten



2 : Connect to the database and list the tables in DB

In [2]:
con = dbConnect(SQLite(), dbname="soccerdatabase.sqlite")
dbListTables(con)

[1] "Country"           "League"            "Match"            
[4] "Player"            "Player_Attributes" "Team"             
[7] "Team_Attributes"   "sqlite_sequence"

Load the tables as dataframe

In [3]:
Country = tbl_df(dbGetQuery(con,"SELECT * FROM Country"))
League = tbl_df(dbGetQuery(con,"SELECT * FROM League"))
Match = tbl_df(dbGetQuery(con,"SELECT * FROM Match"))
Player = tbl_df(dbGetQuery(con,"SELECT * FROM player"))
Player_Attributes = tbl_df(dbGetQuery(con,"SELECT * FROM Player_Attributes"))
Team = tbl_df(dbGetQuery(con,"SELECT * FROM Team"))
Team_Attributes = tbl_df(dbGetQuery(con,"SELECT * FROM Team_Attributes"))

# Q1: Which are the top 10 players by overall rating?


Look into the values of player and player_attributes df

In [4]:
head(Player)
head(Player_Attributes)

id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154
6,27316,Aaron Hunt,158138,1986-09-04 00:00:00,182.88,161


id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
1,218353,505942,2016-02-18 00:00:00,67,71,right,medium,medium,49,...,54,48,65,69,69,6,11,10,8,8
2,218353,505942,2015-11-19 00:00:00,67,71,right,medium,medium,49,...,54,48,65,69,69,6,11,10,8,8
3,218353,505942,2015-09-21 00:00:00,62,66,right,medium,medium,49,...,54,48,65,66,69,6,11,10,8,8
4,218353,505942,2015-03-20 00:00:00,61,65,right,medium,medium,48,...,53,47,62,63,66,5,10,9,7,7
5,218353,505942,2007-02-22 00:00:00,61,65,right,medium,medium,48,...,53,47,62,63,66,5,10,9,7,7
6,189615,155782,2016-04-21 00:00:00,74,76,left,high,medium,80,...,66,59,76,75,78,14,7,9,9,12


4: We need to join the 2 df and retain the player df with ratings in sorted order.

In [5]:
Player=group_by(Player_Attributes,player_api_id)%>%
summarise(max_rating=max(overall_rating))%>%           #find the maximum rating of each player
right_join(Player,by = "player_api_id")%>%             #join the rating temporary df with player df
arrange(desc(max_rating))                              #display
head(Player,n=10)

player_api_id,max_rating,id,player_name,player_fifa_api_id,birthday,height,weight
30981,94,6176,Lionel Messi,158023,1987-06-24 00:00:00,170.18,159
30893,93,1995,Cristiano Ronaldo,20801,1985-02-05 00:00:00,185.42,176
30717,93,3826,Gianluigi Buffon,1179,1978-01-28 00:00:00,193.04,201
30829,93,10749,Wayne Rooney,54050,1985-10-24 00:00:00,175.26,183
39989,92,3994,Gregory Coupet,1747,1972-12-31 00:00:00,180.34,176
39854,92,10861,Xavi Hernandez,10535,1980-01-25 00:00:00,170.18,148
30723,91,388,Alessandro Nesta,1088,1976-03-19 00:00:00,187.96,174
30955,91,742,Andres Iniesta,41,1984-05-11 00:00:00,170.18,150
34520,91,3183,Fabio Cannavaro,1183,1973-09-13 00:00:00,175.26,165
30657,91,4366,Iker Casillas,5479,1981-05-20 00:00:00,185.42,185


# Q2. Which are the top 10 goalkeepers by sum of gk attributes?

In [6]:
# head(Player_Attributes)
# player2=group_by(Player_Attributes,player_api_id)%>%
# melt(Match,id = c(1:11), measure=c(12:33),na.rm = TRUE, value.name = "player_api_id")
# rowSums(Match","gk_diving":"gk_reflexes", na.rm=TRUE)
temp1=select(Player_Attributes, player_api_id,starts_with("gk_"))#select all the gk attributes
Player=mutate(temp1,total_gk=rowSums(temp1[,c(2,3,4,5,6)]))%>%   #sum the gk attributes
group_by(player_api_id)%>%                                       #group by player id to find gk attribute of each player
summarise(max_gk=max(total_gk))%>%                               #Find maximum of gk attributes of a player
arrange(desc(max_gk))%>%                                         #arrange in descending order to retrieve the top 10 players
left_join(Player)                                                #join the gk attribute df with player df
Player=select(Player,player_api_id,max_gk,max_rating,player_name)#display result
head(Player)



Joining, by = "player_api_id"


player_api_id,max_gk,max_rating,player_name
30717,449,93,Gianluigi Buffon
39989,447,92,Gregory Coupet
30859,445,89,Petr Cech
30657,442,91,Iker Casillas
27299,440,90,Manuel Neuer
30989,438,89,Julio Cesar


# Q4. Which are the top 10 players by number of leagues they played in?

In [7]:
#Filter only the required columns from the dataframes
#reference : Kaggle discussion
Country=rename(Country, country_id = id,country_name=name) #Filter country dataframe
League = select(League, country_id, name)                  #Filter League dataframe
League=rename(League,League_name=name)                     #Rename name column to avoid ambiguity
head(Country)
head(League)
Team = select(Team, team_api_id, team_long_name, team_short_name) 
head(Team)
Match = select(Match, id, country_id, league_id, season, stage, date, match_api_id, home_team_api_id, away_team_api_id, home_team_goal, away_team_goal, home_player_1, home_player_2, home_player_3, home_player_4, home_player_5, home_player_6, home_player_7, home_player_8, home_player_9, home_player_10, home_player_11, away_player_1, away_player_2, away_player_3, away_player_4, away_player_5, away_player_6, away_player_7, away_player_8, away_player_9, away_player_10, away_player_11, goal, shoton, shotoff, foulcommit, card, cross, corner, possession)
head(Match)
head(Player)

country_id,country_name
1,Belgium
1729,England
4769,France
7809,Germany
10257,Italy
13274,Netherlands


country_id,League_name
1,Belgium Jupiler League
1729,England Premier League
4769,France Ligue 1
7809,Germany 1. Bundesliga
10257,Italy Serie A
13274,Netherlands Eredivisie


team_api_id,team_long_name,team_short_name
9987,KRC Genk,GEN
9993,Beerschot AC,BAC
10000,SV Zulte-Waregem,ZUL
9994,Sporting Lokeren,LOK
9984,KSV Cercle Brugge,CEB
8635,RSC Anderlecht,AND


id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession
1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,1,2008/2009,1,2008-09-24 00:00:00,492478,8203,8342,1,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


player_api_id,max_gk,max_rating,player_name
30717,449,93,Gianluigi Buffon
39989,447,92,Gregory Coupet
30859,445,89,Petr Cech
30657,442,91,Iker Casillas
27299,440,90,Manuel Neuer
30989,438,89,Julio Cesar


7: Combining all the attributes to match_sublime table <br/>
 Mapping team api id to player api id<br\>
If player is home player , copy away api id else copy home api id.

In [8]:
head(Match[,12:33])
head(Match[,1:11])
match_sublime <-melt(Match,id = c(1:11), measure=c(12:33),na.rm = TRUE, value.name = "player_api_id")  
# 1:11 match attributes and 12:33 id of the 22 players
#melting both so as to represent them in single column
match_sublime=mutate(match_sublime,team_api_id=ifelse(grepl("home",variable),home_team_api_id,
                            ifelse(grepl("away",variable),away_team_api_id,NA)))
head(match_sublime)

home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,...,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1
2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0
3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3
4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0
5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3
6,1,1,2008/2009,1,2008-09-24 00:00:00,492478,8203,8342,1,1


id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,variable,player_api_id,team_api_id
145,1,1,2008/2009,24,2009-02-28 00:00:00,493016,9996,8635,1,1,home_player_1,39890,9996
146,1,1,2008/2009,24,2009-02-27 00:00:00,493017,8203,9987,2,1,home_player_1,38327,8203
147,1,1,2008/2009,24,2009-02-28 00:00:00,493018,9986,9998,3,0,home_player_1,95597,9986
149,1,1,2008/2009,24,2009-03-01 00:00:00,493020,9994,9991,0,1,home_player_1,30934,9994
150,1,1,2008/2009,24,2009-02-28 00:00:00,493021,8342,9999,2,1,home_player_1,37990,8342
151,1,1,2008/2009,24,2009-02-28 00:00:00,493022,9993,8571,3,0,home_player_1,38391,9993


8: Mapping team api id to player api id<br\>
If player is home player , copy away api id else copy home api id.

In [9]:
#Reference: Kaggle discussion
match_sublime_alljoin=left_join(match_sublime,Team, by = "team_api_id")%>%
left_join(Player, by = "player_api_id") %>% # adding team to each player
left_join(Country, by = "country_id") %>% # adding country attributes
left_join(League, by = "country_id") # adding league attributes

In [10]:
tail(match_sublime_alljoin)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,variable,player_api_id,team_api_id,team_long_name,team_short_name,max_gk,max_rating,player_name,country_name,League_name
542276,25974,24558,24558,2015/2016,8,2015-09-13 00:00:00,1992090,10179,7896,3,...,away_player_11,178142,7896,Lugano,LUG,55,62,Mattia Bottani,Switzerland,Switzerland Super League
542277,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,...,away_player_11,289732,10191,FC Thun,THU,45,67,Roman Buess,Switzerland,Switzerland Super League
542278,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,...,away_player_11,95257,10199,FC Luzern,LUZ,145,73,Jakob Jantscher,Switzerland,Switzerland Super League
542279,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,...,away_player_11,275122,10179,FC Sion,SIO,44,73,Moussa Konate,Switzerland,Switzerland Super League
542280,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,...,away_player_11,194532,10243,FC Zürich,ZUR,53,68,Franck Etoundi,Switzerland,Switzerland Super League
542281,25979,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992095,10192,9931,4,...,away_player_11,527103,9931,FC Basel,BAS,45,76,Breel Embolo,Switzerland,Switzerland Super League


9: Solution: Which are the top 10 players by number of leagues they played in?

In [11]:
Player=select(match_sublime_alljoin,League_name,player_api_id)%>% #select league name corresponding to player api id 
group_by(player_api_id,League_name)%>%                            #Group by player_Api_id and League Name
summarize(oneLeaguecount=n())%>%                                  #count by league name
group_by(player_api_id)%>%                                        #group by player api id
summarize(totalLeaguecount=n())%>%                                #count no of leagues played by each player      
right_join(Player, by="player_api_id")%>%
arrange(desc(totalLeaguecount))
head(Player)

player_api_id,totalLeaguecount,max_gk,max_rating,player_name
42119,6,NA,NA,Gelson Fernandes
178538,5,134,77,Filip Djuricic
116750,5,106,77,Ricky van Wolfswinkel
30841,4,424,85,Maarten Stekelenburg
109060,4,392,78,Sergio Romero
38229,4,181,82,Bryan Ruiz


# Q5: Which are the top 10 teams by sum of build up play attributes?

In [12]:
head(Team)
head(Team_Attributes)

team_api_id,team_long_name,team_short_name
9987,KRC Genk,GEN
9993,Beerschot AC,BAC
10000,SV Zulte-Waregem,ZUL
9994,Sporting Lokeren,LOK
9984,KSV Cercle Brugge,CEB
8635,RSC Anderlecht,AND


id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
1,434,9930,2010-02-22 00:00:00,60,Balanced,NA,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
2,434,9930,2014-09-19 00:00:00,52,Balanced,48,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,434,9930,2015-09-10 00:00:00,47,Balanced,41,Normal,54,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
4,77,8485,2010-02-22 00:00:00,70,Fast,NA,Little,70,Long,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
5,77,8485,2011-02-22 00:00:00,47,Balanced,NA,Little,52,Mixed,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover
6,77,8485,2012-02-22 00:00:00,58,Balanced,NA,Little,62,Mixed,...,55,Normal,Organised,40,Medium,40,Press,60,Normal,Cover


In [13]:
TeamBuildUp=select(Team_Attributes,team_api_id,starts_with("buildUp"))%>%#select all column starting with buildUp
na.omit()%>%                                                             #omit na
select_if(is.numeric)                                                    #select only numeric columns
Team=mutate(TeamBuildUp,totalBuildUp=rowSums(TeamBuildUp[,2:4]))%>%      #sum the buildup attributes
select(team_api_id,totalBuildUp)%>%
group_by(team_api_id)%>%
summarize(maxTotalBuildUp=max(totalBuildUp))%>%                          #find the maximum among all build up attribute of each player
right_join(Team,by="team_api_id")%>%
na.omit()

11: Solution: Which are the top 10 teams by sum of build up play attributes?

In [14]:
a=arrange(Team,desc(maxTotalBuildUp))
a[1:10,]

team_api_id,maxTotalBuildUp,team_long_name,team_short_name
9804,209,Torino,TOR
8600,205,Udinese,UDI
8194,198,Fortuna Düsseldorf,FDU
8033,193,Podbeskidzie Bielsko-Biala,POD
8245,192,Korona Kielce,KKI
208931,190,Carpi,CAP
8244,190,Widzew Lódz,LOD
8024,190,Widzew Lódz,WID
9830,189,FC Nantes,NAN
8543,188,Lazio,LAZ


#  Q6. Which are the top 10 teams by sum of chance creation attributes?

In [15]:
# #the same as of buildup
TeamChanceCreation=select(Team_Attributes,team_api_id,starts_with("chanceCreation"))%>%#select all column starting with buildUp
na.omit()%>%#omit na
select_if(is.numeric)                                                                   #select only numeric columns
Team=mutate(TeamChanceCreation,totalChanceCreation=rowSums(TeamChanceCreation[,2:4]))%>%#sum the chance creatuin attributes
select(team_api_id,totalChanceCreation)%>%
group_by(team_api_id)%>%
summarize(maxTotalChanceCreation=max(totalChanceCreation))%>%                            #find max of chance creations of a player
right_join(Team,by="team_api_id")%>%
na.omit()

13: Solution: Which are the top 10 teams by sum of build up play attributes?

In [16]:
b=arrange(Team,desc(maxTotalChanceCreation))
b[1:10,]

team_api_id,maxTotalChanceCreation,maxTotalBuildUp,team_long_name,team_short_name
8543,220,188,Lazio,LAZ
8697,216,174,SV Werder Bremen,WBR
9875,215,180,Napoli,NAP
8456,210,123,Manchester City,MCI
8586,210,133,Tottenham Hotspur,TOT
8658,210,174,Birmingham City,BIR
8602,210,136,Wolverhampton Wanderers,WOL
8191,210,154,Burnley,BUR
10003,210,131,Swansea City,SWA
10172,210,141,Queens Park Rangers,QPR


#  Q7. Which are the top 10 teams by sum of defense attributes??

In [17]:
#the same as of buildup
TeamDefenceAttr=select(Team_Attributes,team_api_id,starts_with("defence"))%>%#select all column starting with buildUp
na.omit()%>%#omit na
select_if(is.numeric)                                                          #select only numeric columns
Team=mutate(TeamDefenceAttr,totalDefenceAttr=rowSums(TeamDefenceAttr[,2:4]))%>%#sum the defense attributes
select(team_api_id,totalDefenceAttr)%>%
group_by(team_api_id)%>%
summarize(maxTotalDefenceAttr=max(totalDefenceAttr))%>%                        #select max of defense attribute
right_join(Team,by="team_api_id")%>%
na.omit()
Team

team_api_id,maxTotalDefenceAttr,maxTotalChanceCreation,maxTotalBuildUp,team_long_name,team_short_name
9987,205,182,148,KRC Genk,GEN
10000,200,153,147,SV Zulte-Waregem,ZUL
9994,200,154,159,Sporting Lokeren,LOK
9984,195,200,145,KSV Cercle Brugge,CEB
8635,190,182,149,RSC Anderlecht,AND
9991,166,184,142,KAA Gent,GEN
9998,138,168,160,RAEC Mons,MON
9985,205,182,161,Standard de Liège,STL
8203,190,150,134,KV Mechelen,MEC
8342,200,170,139,Club Brugge KV,CLB


15: Solution: Which are the top 10 teams by sum of defense attributes??

In [18]:
c=arrange(Team,desc(maxTotalDefenceAttr))
c[1:10,]

team_api_id,maxTotalDefenceAttr,maxTotalChanceCreation,maxTotalBuildUp,team_long_name,team_short_name
10001,210,167,148,KVC Westerlo,WES
9986,210,148,160,Sporting Charleroi,CHA
8659,210,195,145,West Bromwich Albion,WBA
8667,210,195,151,Hull City,HUL
8658,210,210,174,Birmingham City,BIR
8602,210,210,136,Wolverhampton Wanderers,WOL
8483,210,205,183,Blackpool,BLA
10003,210,210,131,Swansea City,SWA
10172,210,210,141,Queens Park Rangers,QPR
8466,210,205,129,Southampton,SOU


#  Q8. Which are the top 10 teams by number of scored goals?

In [19]:
homeTeamGoal=select(Match,home_team_api_id,home_team_goal)%>%#select home team column
group_by(home_team_api_id)%>%
summarize(totalGoals=sum(home_team_goal))%>% #sum goals by same team api id
rename(team_api_id=home_team_api_id) #rename column for rbind
awayTeamGoal=select(Match,away_team_api_id,away_team_goal)%>%#select away team column
group_by(away_team_api_id)%>%
summarize(totalGoals=sum(away_team_goal))%>%#sum goals by same team api id
rename(team_api_id=away_team_api_id)#rename column for rbind
Team=rbind(homeTeamGoal,awayTeamGoal)%>%#stack  homeTeamGoal and awayTeamGoal on top of each other
group_by(team_api_id)%>%#group similar team together
summarize(totalGoals=sum(totalGoals))%>%#sum up goals by same team id
right_join(Team,by="team_api_id")#join the data with Team dataframe

15: Solution: Which are the top 10 teams by number of scored goals?¶

In [20]:
arrange(Team,desc(totalGoals))

team_api_id,totalGoals,maxTotalDefenceAttr,maxTotalChanceCreation,maxTotalBuildUp,team_long_name,team_short_name
8634,849,195,175,122,FC Barcelona,BAR
8633,843,178,200,153,Real Madrid CF,REA
9925,695,200,205,180,Celtic,CEL
9823,653,184,205,97,FC Bayern Munich,BMU
8640,652,167,200,153,PSV,PSV
8593,647,193,199,117,Ajax,AJA
9931,619,180,186,170,FC Basel,BAS
8456,606,180,210,123,Manchester City,MCI
8455,583,168,200,157,Chelsea,CHE
10260,582,163,200,134,Manchester United,MUN


### Q3 Which are the top 10 players by number of appearances (with any team they played with)?

16: In match_sublime dataframe , we had molten down all the player columns [12:33] to player_Api_id column. Also, in the same data frame we have the match api id of each players. Grouping the players by player api id and counting their match api id will give the number of matches each played irrespective of teams.

In [21]:
head(match_sublime)

id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,variable,player_api_id,team_api_id
145,1,1,2008/2009,24,2009-02-28 00:00:00,493016,9996,8635,1,1,home_player_1,39890,9996
146,1,1,2008/2009,24,2009-02-27 00:00:00,493017,8203,9987,2,1,home_player_1,38327,8203
147,1,1,2008/2009,24,2009-02-28 00:00:00,493018,9986,9998,3,0,home_player_1,95597,9986
149,1,1,2008/2009,24,2009-03-01 00:00:00,493020,9994,9991,0,1,home_player_1,30934,9994
150,1,1,2008/2009,24,2009-02-28 00:00:00,493021,8342,9999,2,1,home_player_1,37990,8342
151,1,1,2008/2009,24,2009-02-28 00:00:00,493022,9993,8571,3,0,home_player_1,38391,9993


17:

In [22]:
Player=select(match_sublime,match_api_id,player_api_id)%>%   #select match id and player id from molten dataframe
group_by(player_api_id)%>%                                   #group by each player
summarize(totalMatchPlayed=n())%>%                           #find number of match played by each player
right_join(Player,by="player_api_id")                        #join with the player table
# arrange(Player,desc(totalMatchPlayed))%>%head(10)

18: Solution Q3

In [27]:
arrange(Player,desc(totalMatchPlayed))%>%head(10)%>%select(player_name,totalMatchPlayed)

player_name,totalMatchPlayed
Steve Mandanda,300
Stephane Ruffier,294
Samir Handanovic,286
Gorka Iraizoz Moreno,286
Hugo Lloris,282
Tim Howard,282
Joe Hart,275
Cedric Carrasso,268
Daniel Congre,262
Moussa Sissoko,260


# Write the data to .csv files

In [23]:
write.csv(Team, "TeamData.csv",na="")
write.csv(Player, "PlayerData.csv",na="")

In [28]:
head(Team)
head(Player)

team_api_id,totalGoals,maxTotalDefenceAttr,maxTotalChanceCreation,maxTotalBuildUp,team_long_name,team_short_name
9987,348,205,182,148,KRC Genk,GEN
10000,306,200,153,147,SV Zulte-Waregem,ZUL
9994,279,200,154,159,Sporting Lokeren,LOK
9984,213,195,200,145,KSV Cercle Brugge,CEB
8635,427,190,182,149,RSC Anderlecht,AND
9991,390,166,184,142,KAA Gent,GEN


player_api_id,totalMatchPlayed,totalLeaguecount,max_gk,max_rating,player_name
42119,175,6,NA,NA,Gelson Fernandes
178538,105,5,134,77,Filip Djuricic
116750,151,5,106,77,Ricky van Wolfswinkel
30841,147,4,424,85,Maarten Stekelenburg
109060,101,4,392,78,Sergio Romero
38229,167,4,181,82,Bryan Ruiz
